# Create ground truth dataset

code is based on the idea of this paper: https://www.sciencedirect.com/science/article/pii/S136481522400197X#sec3

In [97]:
#imports
import pandas as pd
import os
import numpy as np

In [98]:
pd.set_option('display.max_columns', None)
data = pd.read_csv('C:/Users/Ulli/Documents/GitHub/MasterThesis_FeatureExtraction_RemoteSensing/data/IFMHA.csv')
data.dropna(inplace=True) #drop nan values
data = data.reset_index(drop=True) #reset index after dropping rows

C:\Users\Ulli\AppData\Local\Temp\ipykernel_24556\2654368737.py:2: DtypeWarning: Columns (4,8,37) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('C:/Users/Ulli/Documents/GitHub/MasterThesis_FeatureExtraction_RemoteSensing/data/IFMHA.csv')


In [99]:
len(data)

124824

In [100]:
data.head()

,site_no,COMID,DASqKm,DASqMi,FTYPE,LatSite,LonSite,SLOPE,STATE,STATE_CD,StreamOrde,chan_area,chan_discharge,chan_evenness,chan_loc_cd,chan_loc_dist,chan_material,chan_name,chan_nu,chan_stability,chan_velocity,chan_width,contrib_drain_area_va,control_type_cd,dec_lat_va,dec_long_va,discharge_cd,discharge_va,drain_area_va,gage_height_va,gage_va_change,gage_va_time,horz_vel_desc,long_vel_desc,meas_type,measured_rating_diff,measurement_dt,measurement_nu,party_nm,q_meas_used_fg,site_tp_cd,site_visit_coll_agency_cd,station_nm,streamflow_method,tz_cd,velocity_method,vert_vel_desc
0,1010000,4289887.0,3473.19,1341.0,ArtificialPath,46.700556,-69.715556,0.003156,ME,23.0,4.0,652.0,1190.0,UNEV,DNST,100.0,BLDR,Main,1.0,FIRM,1.83,350.0,1341.0,IceCover,46.700556,-69.715556,NONE,1190.0,1341.0,3.89,0.00,0.5,UNEV,unkn,CWAY,Poor,2009-01-06 14:49:00,365,LEF/ALU,Yes,ST,USGS,"St. John River at Ninemile Bridge, Maine",QADCP,EST,VADCP,UNSP
1,1010000,4289887.0,3473.19,1341.0,ArtificialPath,46.700556,-69.715556,0.003156,ME,23.0,4.0,386.0,378.0,UNEV,DNST,75.0,BLDR,Main,1.0,FIRM,0.98,348.0,1341.0,IceCover,46.700556,-69.715556,NONE,378.0,1341.0,2.77,0.00,0.8,UNEV,unkn,MBOT,Poor,2009-02-03 09:37:17,366,LHC/NWS,Yes,ST,USGS,"St. John River at Ninemile Bridge, Maine",QADCP,EST,VADCP,UNSP
2,1010000,4289887.0,3473.19,1341.0,ArtificialPath,46.700556,-69.715556,0.003156,ME,23.0,4.0,4620.0,27100.0,EVEN,UPST,4000.0,BLDR,Main,1.0,FIRM,5.87,466.0,1341.0,Clear,46.700556,-69.715556,NONE,27100.0,1341.0,9.38,-0.01,0.3,EVEN,unkn,CWAY,Fair,2009-04-24 09:34:30,367,NWS/MAH,Yes,ST,USGS,"St. John River at Ninemile Bridge, Maine",QADCP,EDT,VADCP,UNSP
3,1010000,4289887.0,3473.19,1341.0,ArtificialPath,46.700556,-69.715556,0.003156,ME,23.0,4.0,2300.0,5050.0,UNEV,UPST,3000.0,BLDR,MAIN,1.0,FIRM,2.19,448.0,1341.0,Clear,46.700556,-69.715556,NONE,5050.0,1341.0,4.29,-0.05,1.2,EVEN,unkn,CWAY,Fair,2009-05-13 16:16:30,368,ARC/MAH,Yes,ST,USGS,"St. John River at Ninemile Bridge, Maine",QSCMM,EDT,VPAA,UNSP
4,1010000,4289887.0,3473.19,1341.0,ArtificialPath,46.700556,-69.715556,0.003156,ME,23.0,4.0,607.0,512.0,UNEV,UPST,1500.0,BLDR,Main,1.0,FIRM,0.84,310.0,1341.0,Clear,46.700556,-69.715556,NONE,593.0,1341.0,1.59,-0.01,1.0,EVEN,unkn,WADE,Fair,2009-08-19 14:08:00,369,ARC/MAH,Yes,ST,USGS,"St. John River at Ninemile Bridge, Maine",QSCMM,EDT,VPAA,UNSP


In [101]:
subset = data[['site_no', 'STATE', 'party_nm', 'station_nm', 'measurement_dt', 'tz_cd', 'measured_rating_diff', 'LatSite', 'LonSite', 'control_type_cd', 'chan_area', 'chan_evenness', 'chan_width', 'SLOPE', 'gage_height_va', 'gage_va_change', 'chan_velocity', 'velocity_method', 'chan_discharge', 'discharge_va', 'streamflow_method' ]] 
subset = subset[subset["measured_rating_diff"].str.contains("Poor") == False] #remove poor quality of the measurement
subset = subset[subset['measured_rating_diff'].str.contains('Unspecified') == False] #remove unspecified quality of the measurement
subset = subset[subset['control_type_cd'].str.contains('IceCover') == False] #remove rows when ice cover is present at time of the measurement

In [102]:
len(subset)

102373

In [103]:
#calculate mean depth
def calculate_mean_depth(cross_section_area, channel_width):
    """
    calculate the mean depth of the river based on cross-sectional area and channel width

    Args:
        cross_section_area (float): the cross-sectional area 
        channel_width (float): the width

    Returns: the mean depth as float. 
             returns 0.0, if width is 0 to avoid division through 0
    """
   #check if channel width is 0
    if channel_width == 0:
        return 0.0
    
    mean_depth = cross_section_area / channel_width
    return mean_depth

In [104]:
def calculate_max_depth(mean_depth):
    """
    calculate hte max depth based on mean_depth and the defined parameters of the LOC-regression from the paper.
    
    Args:
        mean_depth: mean depth calculated above

    Returns: estimated max depth as float
    """
    
    # calibratet parameters from LOC-regression (chapter 4.3. cross-section)
    alpha = 1.48 #slope
    beta = -0.01 #intercept
    
    # equation from the paper
    max_depth = (alpha * mean_depth) + beta
    
    # if depth is negative, set it to 0
    if max_depth < 0:
        return 0.0
    
    return max_depth

In [105]:
#add new columns to dataset 
def add_depth_columns(df):
    """
    add mean depth and max depth columns to the dataframe

    Args:
        df: pd dataframe

    """
    mean_depths = []
    max_depths = []
    
    for i, row in df.iterrows():
        cross_section_area = row['chan_area']
        channel_width = row['chan_width']
        
        mean_depth = calculate_mean_depth(cross_section_area, channel_width)
        max_depth = calculate_max_depth(mean_depth)
        
        mean_depths.append(mean_depth)
        max_depths.append(max_depth)
    
    df['mean_depth'] = mean_depths
    df['max_depth'] = max_depths
    
    return df

In [106]:
subset = add_depth_columns(subset)

In [107]:
#-- Code from paper from github repository: https://github.com/smhassanerfani/ifmha/blob/main/ifmha_geometry_analysis.ipynb --
# calculate wetted perimeter
# paper estimated parameters for two ideal cross-sections (trapezoidal and parabolic)
import time
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter

import seaborn as sns
sns.set(style='darkgrid')

from scipy.integrate import quad
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

plt.rcParams.update({
    # 'font.sans-serif': 'Comic Sans MS',
    'font.family': 'serif',
    # 'font.size'   : 40
})

%load_ext autoreload
%autoreload 2

ifmha = subset.copy()
candidate_columns = ['chan_width', 'chan_discharge', 'chan_area', 'SLOPE','max_depth']
ifmha = ifmha[(ifmha[candidate_columns] > 0).all(axis=1)]
print(ifmha.shape)

# Trapezoid Cross-section
t1 = time.time()

# Side Slope (Ss):
ifmha_ss = ifmha.groupby("site_no").apply(lambda x: LinearRegression(positive=True).fit(x['max_depth'].values.reshape(-1, 1), 0.5 * x['chan_width'].values.reshape(-1, 1)).coef_[0,0]).reset_index(drop=False)

# Bottom Width (Bw):
ifmha_bw =  ifmha.groupby("site_no").apply(lambda x: LinearRegression(positive=True).fit(x['max_depth'].values.reshape(-1, 1), 0.5 * x['chan_width'].values.reshape(-1, 1)).intercept_[0] * 2.0).reset_index(drop=False)

# Append Trapezoid Cross-section Parameters to IFMHA
ifmha = pd.merge(ifmha, ifmha_ss, how="left", on=["site_no"])
ifmha = pd.merge(ifmha, ifmha_bw, how="left", on=["site_no"])
print(time.time() - t1)



# Parabola Cross-section
t1 = time.time()
# Parabola Coefficient (a):
ifmha_a = ifmha.groupby("site_no").apply(lambda x: LinearRegression(fit_intercept=False).fit((0.5 * x['chan_width']**2).values.reshape(-1, 1), x['max_depth'].values.reshape(-1, 1)).coef_[0,0]).reset_index(drop=False)
# Append Parabola Cross-section Coefficient to IFMHA
ifmha = pd.merge(ifmha, ifmha_a, how="left", on=["site_no"])
print(time.time() - t1)

ifmha.rename(columns={'0_x': 'side_slope', '0_y': 'chan_bottom_width', 0:'parabola_coeff'}, inplace=True)

# Define the function for the parabolic curve y = ax^2
def parabola(x, a):
    return a * x**2

# Define the derivative of the parabolic curve
def derivative(x, a):
    return 2 * a * x

# Define the function for the integrand to calculate arc length
def integrand(x, a):
    return np.sqrt(1 + derivative(x, a)**2)

def cal_arc_length(func, x_min, x_max, a):
    # Calculate the arc length using quad (numerical integration)
    arc_length, _ = func(integrand, x_min, x_max, args=(a,))
    return arc_length


ifmha['arc_length'] = ifmha.apply(lambda x: cal_arc_length(quad, -x['chan_width'] / 2, x['chan_width'] / 2, x['parabola_coeff']), axis=1)

# Exclude negative bottom width and steep side slope:
ifmha = ifmha[(ifmha['side_slope'] <= 12.0) & (ifmha['chan_bottom_width'] >= 0.0)]

# Estimate wetted perimeter based on Trapezoid cross-section
ifmha.loc[:, 'wetted_perimeter'] = ifmha['chan_bottom_width'] + (2 * ifmha['max_depth'] * (1 + ifmha['side_slope']**2)**0.5)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
(102196, 23)


C:\Users\Ulli\AppData\Local\Temp\ipykernel_24556\1382731332.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ifmha_ss = ifmha.groupby("site_no").apply(lambda x: LinearRegression(positive=True).fit(x['max_depth'].values.reshape(-1, 1), 0.5 * x['chan_width'].values.reshape(-1, 1)).coef_[0,0]).reset_index(drop=False)
C:\Users\Ulli\AppData\Local\Temp\ipykernel_24556\1382731332.py:39: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to s

5.340480089187622


C:\Users\Ulli\AppData\Local\Temp\ipykernel_24556\1382731332.py:51: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ifmha_a = ifmha.groupby("site_no").apply(lambda x: LinearRegression(fit_intercept=False).fit((0.5 * x['chan_width']**2).values.reshape(-1, 1), x['max_depth'].values.reshape(-1, 1)).coef_[0,0]).reset_index(drop=False)


1.767193078994751


In [109]:
len(ifmha)

93694

In [110]:
#calculate manning n
def calculate_manning_n(area, wetted_perimeter, slope, discharge):
    """
    Calculate Manning's n using the Manning equation rearranged to solve for n.

    Args:
        area (float): Cross-sectional area of flow (A)
        wetted_perimeter (float): Wetted perimeter (P)
        slope (float): Channel slope (S)
        discharge (float): Discharge (Q)
    Returns:
        float: Calculated Manning's n value

    """
    if area <= 0 or wetted_perimeter <= 0 or slope <= 0 or discharge <= 0:
        return np.nan  # Return NaN for invalid inputs to avoid division by zero or negative values

    hydraulic_radius = area / wetted_perimeter  # R = A / P
    manning_n = (area * (hydraulic_radius ** (2/3)) * (slope ** (1/2))) / discharge  # n = (A * R^(2/3) * S^(1/2)) / Q
    return manning_n

In [111]:
#create whole dataset
ifmha['manning_n'] = ifmha.apply(lambda x: calculate_manning_n(x['chan_area'], x['wetted_perimeter'], x['SLOPE'], x['chan_discharge']), axis=1)

In [ ]:
#create a column for year to filter just for years after 2015 (data availabe after 04.07.2015)
ifmha['year'] = ifmha['measurement_dt'].apply(lambda x: x.split(' ')[0].split('-')[0]).astype(int)
ifmha['month'] = ifmha['measurement_dt'].apply(lambda x: x.split(' ')[0].split('-')[1]).astype(int)
ifmha['day'] = ifmha['measurement_dt'].apply(lambda x: x.split(' ')[0].split('-')[2]).astype(int)
ifmha = ifmha[ifmha['year'] >= 2016]

In [113]:
ifmha.to_csv('C:/Users/Ulli/Documents/GitHub/MasterThesis_FeatureExtraction_RemoteSensing/data_new/IFMHA_ground_truth.csv', index=False)

In [ ]:
ifmha_1010070 = ifmha[ifmha['site_no'] == 1010070] #filter for just one river

In [115]:
ifmha_1010070

,site_no,STATE,party_nm,station_nm,measurement_dt,tz_cd,measured_rating_diff,LatSite,LonSite,control_type_cd,chan_area,chan_evenness,chan_width,SLOPE,gage_height_va,gage_va_change,chan_velocity,velocity_method,chan_discharge,discharge_va,streamflow_method,mean_depth,max_depth,side_slope,chan_bottom_width,parabola_coeff,arc_length,wetted_perimeter,manning_n,year,month,day
104,1010070,ME,JRC/SGS,"Big Black River near Depot Mtn, Maine",2016-05-24 08:14:30,EDT,Good,46.893889,-69.751667,Clear,100.0,EVEN,84.0,0.001153,3.95,-0.01,1.33,VADV,133.0,133.0,QSCMM,1.190476,1.751905,1.523489,76.409081,0.001022,84.103050,82.794305,0.028954,2016,5,24
105,1010070,ME,JRC,"Big Black River near Depot Mtn, Maine",2016-09-08 08:43:00,EDT,Fair,46.893889,-69.751667,Clear,55.0,EVEN,75.0,0.001153,3.07,0.00,0.32,VADV,17.5,17.5,QSCMM,0.733333,1.075333,1.523489,76.409081,0.001022,75.073365,80.328383,0.082900,2016,9,8
106,1010070,ME,JRC,"Big Black River near Depot Mtn, Maine",2016-10-24 07:24:05,EDT,Good,46.893889,-69.751667,Clear,821.0,EVEN,113.0,0.001153,9.96,-0.03,3.93,VADCP,3230.0,3230.0,QADCP,7.265487,10.742920,1.523489,76.409081,0.001022,113.250645,115.564148,0.031895,2016,10,24
107,1010070,ME,JRP/ALU,"Big Black River near Depot Mtn, Maine",2017-04-18 14:52:09,EDT,Fair,46.893889,-69.751667,Clear,654.0,EVEN,105.0,0.001153,8.26,0.03,3.09,VADCP,2020.0,2020.0,QADCP,6.228571,9.208286,1.523489,76.409081,0.001022,105.201145,109.970815,0.036085,2017,4,18
108,1010070,ME,JRC/HSG,"Big Black River near Depot Mtn, Maine",2017-05-03 08:37:23,EDT,Good,46.893889,-69.751667,Clear,987.0,UNEV,121.0,0.001153,11.19,0.00,4.48,VADCP,4430.0,4430.0,QADCP,8.157025,12.062397,1.523489,76.409081,0.001022,121.307648,120.373285,0.030761,2017,5,3
109,1010070,ME,JRC/SGS,"Big Black River near Depot Mtn, Maine",2017-06-02 08:54:13,EDT,Good,46.893889,-69.751667,Clear,382.0,UNEV,96.8,0.001153,5.46,-0.01,1.43,VADCP,547.0,547.0,QADCP,3.946281,5.830496,1.523489,76.409081,0.001022,96.957645,97.659677,0.058867,2017,6,2
110,1010070,ME,JRC,"Big Black River near Depot Mtn, Maine",2017-07-25 13:48:30,EDT,Fair,46.893889,-69.751667,Clear,85.8,UNEV,84.0,0.001153,3.34,0.00,0.49,VADV,41.8,41.8,QSCMM,1.021429,1.501714,1.523489,76.409081,0.001022,84.103050,81.882427,0.071902,2017,7,25
111,1010070,ME,JRC,"Big Black River near Depot Mtn, Maine",2017-09-19 16:08:30,EDT,Fair,46.893889,-69.751667,Clear,66.6,UNEV,78.5,0.001153,3.09,0.00,0.29,VADV,19.3,19.3,QSCMM,0.848408,1.245643,1.523489,76.409081,0.001022,78.584116,80.949117,0.102878,2017,9,19
112,1010070,ME,JRC/JRP,"Big Black River near Depot Mtn, Maine",2018-05-03 14:12:53,EDT,Good,46.893889,-69.751667,Clear,1030.0,UNEV,122.0,0.001153,11.46,-0.01,4.56,VADCP,4670.0,4690.0,QADCP,8.442623,12.485082,1.523489,76.409081,0.001022,122.315327,121.913860,0.031065,2018,5,3
113,1010070,ME,JRC/HSG,"Big Black River near Depot Mtn, Maine",2018-05-15 15:25:58,EDT,Good,46.893889,-69.751667,Clear,352.0,UNEV,93.8,0.001153,5.10,0.00,1.29,VADCP,454.0,454.0,QADCP,3.752665,5.543945,1.523489,76.409081,0.001022,93.943450,96.615274,0.062333,2018,5,15
